In [1]:
!pip install tenseal

In [2]:
import tenseal as ts

context=ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
context.generate_galois_keys()
context.global_scale=2**40

In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
# X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = pd.read_csv('X_train3.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train3.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test3.csv')
y_test = pd.read_csv('y_test3.csv')
y_test=y_test.to_numpy().reshape(-1)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [5]:
len(X_train)

1080

In [6]:
len(X_test)

270

In [7]:
enc_X_train=[]
for i in X_train:
  p=ts.ckks_vector(context,i)
  enc_X_train.append(p)

In [8]:
enc_y_train=[]
for i in y_train:
  p=ts.ckks_vector(context,[i])
  enc_y_train.append(p)

In [9]:
def sigmoid(z):
  z=z.decrypt()
  z=round(z[0],7)
  x=1/(1+np.exp(-z))
  x=round(x,7)
  x=ts.ckks_vector(context,[x])
  return x

In [10]:
def relinearlizing_vector(vec):
  vec_2=[]
  vec=vec.decrypt()
  for i in vec:
    p=round(i,7)
    vec_2.append(p)
  vec_2=ts.ckks_vector(context,vec_2)
  return vec_2

In [11]:
def relinearlizing_scalar(sca):
  sca=sca.decrypt()
  sca=round(sca[0],7)
  sca=ts.ckks_vector(context,[sca])
  return sca

In [12]:
n_samples,n_attributes=X_train.shape
n_samples,n_attributes

(1080, 15)

In [13]:
weights=np.zeros(n_attributes)
bias=0
lr=0.01
weights,bias

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 0)

In [14]:
weights=ts.ckks_vector(context,weights)
bias=ts.ckks_vector(context,[bias])

In [15]:
ar=np.arange(n_samples)
for epoch in range(3):
  np.random.shuffle(ar)
  for i in ar:
    xi=enc_X_train[i]
    yi=enc_y_train[i]
    lin_op=xi.dot(weights)
    lin_op=lin_op+bias
    y_pred=sigmoid(lin_op)
    db=y_pred-yi
    dw=db*xi
    dw=relinearlizing_vector(dw)
    db=relinearlizing_scalar(db)
    weights=relinearlizing_vector(weights)
    bias=relinearlizing_scalar(bias)
    weights=weights-(lr*dw)
    bias=bias-(lr*db)
    print(f'epoch={epoch}, {i} loop is executed')

epoch=0, 113 loop is executed
epoch=0, 587 loop is executed
epoch=0, 626 loop is executed
epoch=0, 492 loop is executed
epoch=0, 366 loop is executed
epoch=0, 847 loop is executed
epoch=0, 601 loop is executed
epoch=0, 750 loop is executed
epoch=0, 676 loop is executed
epoch=0, 886 loop is executed
epoch=0, 164 loop is executed
epoch=0, 158 loop is executed
epoch=0, 671 loop is executed
epoch=0, 632 loop is executed
epoch=0, 528 loop is executed
epoch=0, 549 loop is executed
epoch=0, 698 loop is executed
epoch=0, 823 loop is executed
epoch=0, 520 loop is executed
epoch=0, 848 loop is executed
epoch=0, 704 loop is executed
epoch=0, 150 loop is executed
epoch=0, 880 loop is executed
epoch=0, 222 loop is executed
epoch=0, 863 loop is executed
epoch=0, 830 loop is executed
epoch=0, 792 loop is executed
epoch=0, 971 loop is executed
epoch=0, 210 loop is executed
epoch=0, 919 loop is executed
epoch=0, 9 loop is executed
epoch=0, 500 loop is executed
epoch=0, 307 loop is executed
epoch=0, 503

In [16]:
weights.decrypt()

[0.32658321065951845,
 0.6226166087460597,
 -0.17805735877552092,
 0.04062566669967828,
 0.10290046181980983,
 -0.10516074258098713,
 0.11389753525659083,
 0.11550149966200118,
 0.012741444769835464,
 -0.019512713254457206,
 0.3147111170181994,
 0.07676180694944607,
 0.10245262359538901,
 -0.036960472063581204,
 0.1566242957996574]

In [17]:
bias.decrypt()

[-1.8871955018237736]

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
model = LogisticRegression(max_iter=3, solver='saga',C=100)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=100, max_iter=3, solver='saga')

In [20]:
weights_sklearn = model.coef_[0]
bias_sklearn = model.intercept_[0]

In [21]:
print("Weights from scikit-learn:", weights_sklearn)
print("Bias from scikit-learn:", bias_sklearn)

Weights from scikit-learn: [ 0.35644556  0.61615874 -0.09604444  0.12458004  0.10102144 -0.15538946
  0.06171142  0.00100568  0.02206579 -0.01168378  0.37345404  0.03338851
  0.06802111  0.01396828  0.06755725]
Bias from scikit-learn: -2.0674061685232137


In [22]:
print("\nComparison of Weights and Bias:")
print("Weights difference:", np.abs(weights.decrypt() - weights_sklearn))
print("Bias difference:", np.abs(bias.decrypt() - bias_sklearn))


Comparison of Weights and Bias:
Weights difference: [0.02986235 0.00645787 0.08201292 0.08395437 0.00187902 0.05022872
 0.05218612 0.11449582 0.00932435 0.00782894 0.05874292 0.04337329
 0.03443152 0.05092875 0.08906705]
Bias difference: [0.18021067]


In [23]:
enc_X_test=[]
for i in X_test:
  p=ts.ckks_vector(context,i)
  enc_X_test.append(p)

In [24]:
y_pred_enc=[]
for i in enc_X_test:
  lin_op=i.dot(weights)
  lin_op=lin_op+bias
  y_pred=sigmoid(lin_op)
  y_pred_enc.append(y_pred)

In [25]:
y_pred_unenc=[]
for i in y_pred_enc:
  p=i.decrypt()[0]
  if p>=0.5:
    y_pred_unenc.append(1)
  else:
    y_pred_unenc.append(0)

In [26]:
for i in y_pred_enc:
  print(i.decrypt())

[0.04469640061390977]
[0.15558620182712668]
[0.13767439739157414]
[0.04074999965789628]
[0.15496720040647782]
[0.051327499350981215]
[0.11115920006261518]
[0.2500229985722352]
[0.15006499885683966]
[0.25707470011738265]
[0.028298098749523725]
[0.02931709867219361]
[0.02794700128043174]
[0.06089220164109605]
[0.1725794004562652]
[0.21918290015014508]
[0.09932389950586548]
[0.0955282998815701]
[0.2251973977590439]
[0.17290369850112508]
[0.15476850064261188]
[0.12538409456153599]
[0.1344880998688632]
[0.07294659675779236]
[0.23274550033275176]
[0.21286509909464651]
[0.4848196983391497]
[0.5163582956358311]
[0.3272602999129971]
[0.06647289879837477]
[0.27763839962116954]
[0.18409160148556722]
[0.11097109807289472]
[0.23632410120984013]
[0.29735429851596806]
[0.025374001275216666]
[0.21310930240806786]
[0.05342509767194213]
[0.06015129918865481]
[0.2768658003240955]
[0.033530101923023733]
[0.10928940242698147]
[0.19968490233743885]
[0.03677129821759965]
[0.07860499944431441]
[0.028731202785

In [27]:
y_pred=model.predict(X_test)

In [28]:
len(y_pred_unenc)

270

In [29]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8481481481481481


In [30]:
accuracy = accuracy_score(y_test, y_pred_unenc)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8333333333333334
